# Imports

In [ ]:
from multisensory import (
    ClassicalTask,
    BalancedComodulationTask,
    DetectionTask,
    MAPClassifier,
)
import pickle
from pathlib import Path

root = Path(".")
save_path = root / "ideal_data"


# Hyperparameters

In [ ]:
# Tasks
tasks = [
    ClassicalTask(s=0.1),
    DetectionTask(pm=2 / 3, pe=0.3, pc=0.28, pn=1 / 3, pi=0.01),  # dense
    DetectionTask(pm=2 / 3, pe=0.057, pc=0.95, pn=1 / 3, pi=0.01),  # sparse
    BalancedComodulationTask(s=0.2),
]
task_keys = ["classical", "detection_dense", "detection_sparse", "comod+"]
nb_trials = 100000
nb_steps = 90

classifier_type = MAPClassifier
pairs = True  # FtA


# Analysis

In [ ]:
ideal_dict = {}
for a, task in enumerate(tasks):
    classifier = classifier_type(task, pairs=pairs)
    trials_prior = task.generate_trials(nb_trials, nb_steps)
    trials = task.generate_trials(nb_trials, nb_steps)
    trials.M = trials_prior.M

    accs_tmp = []
    for nb_step in range(nb_steps):
        trials.A = trials_prior.A[:, :nb_step]
        trials.V = trials_prior.V[:, :nb_step]
        res = classifier.test(trials)
        accs_tmp.append(res.accuracy)

    ideal_dict[task_keys[a]] = accs_tmp

outfile = open(save_path, "wb")
pickle.dump(ideal_dict, outfile)
outfile.close()
